<a href="https://colab.research.google.com/github/DowookKim/Deep_learing/blob/master/bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
import pandas as pd
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
result = tokenizer.tokenize("Here is the sentence I want embeddings for")
print(result)

['here', 'is', 'the', 'sentence', 'i', 'want', 'em', '##bed', '##ding', '##s', 'for']


In [ ]:
print(tokenizer.vocab['here'])

2182


In [ ]:
print(tokenizer.vocab['embeddings'])


KeyError: 'embeddings'

In [ ]:
print(tokenizer.vocab['em'])
print(tokenizer.vocab['##bed'])
print(tokenizer.vocab['##ding'])
print(tokenizer.vocab['##s'])


7861
8270
4667
2015


In [ ]:
# BERT의 단어 집합을 vocabulary.txt에 저장
with open('vocabulary.txt', 'w') as f:
  for token in tokenizer.vocab.keys():
    f.write(token + '\n')

In [ ]:
df = pd.read_fwf('vocabulary.txt', header=None)
df


,0
0,[PAD]
1,[unused0]
2,[unused1]
3,[unused2]
4,[unused3]
...,...
30517,##．
30518,##／
30519,##：
30520,##？


In [ ]:
print('단어 집합의 크기 :',len(df))
df.loc[4667].values[0]


단어 집합의 크기 : 30522


'##ding'

# Bert 모델 불러와서 써보기


In [ ]:
from transformers import TFBertForMaskedLM
from transformers import AutoTokenizer
#BERT를 마스크드 언어 모델 형태로 로드하기 위한 import

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
model = AutoModelForMaskedLM.from_pretrained("bert-large-uncased")
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased")
'''Google이 공개한
BERT-large (24-layer, 1024 hidden)
Wikipedia + BooksCorpus로
MLM (+ NSP) 프리트레이닝만 수행한 가중치
버트 모델'''
#AutoTokenizer.from_pretrained('모델 이름')을 넣으면 해당 모델이 학습되었을 당시에 사용되었던 토크나이저를 로드

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
inputs = tokenizer(
    "Soccer is a really fun [MASK].",
    return_tensors="pt"
)
#마스크드 언어 모델은 [MASK]의 위치에 해당하는 단어를 예측

In [ ]:
print(inputs['input_ids'])

tensor([[ 101, 4715, 2003, 1037, 2428, 4569,  103, 1012,  102]])


In [ ]:
print(inputs['token_type_ids'])
#크나이저로 변환된 결과에서 token_type_ids를 통해서 문장을 구분하는 세그먼트 인코딩 결과를 확인
#현재의 입력은 문장이 두 개가 아니라 한 개이므로 여기서는 문장 길이만큼의 0 시퀀스

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0]])


In [ ]:
print(inputs['attention_mask'])
#현재의 입력에서는 패딩이 없으므로 여기서는 문장 길이만큼의 1 시퀀스를

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])


In [ ]:
from transformers import FillMaskPipeline
pip = FillMaskPipeline(model=model, tokenizer=tokenizer)


Device set to use cpu


In [ ]:
pip('Soccer is a really fun [MASK].')


[{'score': 0.7621225714683533,
  'token': 4368,
  'token_str': 'sport',
  'sequence': 'soccer is a really fun sport.'},
 {'score': 0.20342221856117249,
  'token': 2208,
  'token_str': 'game',
  'sequence': 'soccer is a really fun game.'},
 {'score': 0.012208708561956882,
  'token': 2518,
  'token_str': 'thing',
  'sequence': 'soccer is a really fun thing.'},
 {'score': 0.0018630452686920762,
  'token': 4023,
  'token_str': 'activity',
  'sequence': 'soccer is a really fun activity.'},
 {'score': 0.0013354998081922531,
  'token': 2492,
  'token_str': 'field',
  'sequence': 'soccer is a really fun field.'}]

In [ ]:
pip('The Avengers is a really fun [MASK].')


[{'score': 0.25629162788391113,
  'token': 2265,
  'token_str': 'show',
  'sequence': 'the avengers is a really fun show.'},
 {'score': 0.17284215986728668,
  'token': 3185,
  'token_str': 'movie',
  'sequence': 'the avengers is a really fun movie.'},
 {'score': 0.11107756197452545,
  'token': 2466,
  'token_str': 'story',
  'sequence': 'the avengers is a really fun story.'},
 {'score': 0.07249030470848083,
  'token': 2186,
  'token_str': 'series',
  'sequence': 'the avengers is a really fun series.'},
 {'score': 0.07046662271022797,
  'token': 2143,
  'token_str': 'film',
  'sequence': 'the avengers is a really fun film.'}]

In [ ]:
pip('I went to [MASK] this morning.')

[{'score': 0.3573085367679596,
  'token': 2147,
  'token_str': 'work',
  'sequence': 'i went to work this morning.'},
 {'score': 0.23304520547389984,
  'token': 2793,
  'token_str': 'bed',
  'sequence': 'i went to bed this morning.'},
 {'score': 0.12845049798488617,
  'token': 2082,
  'token_str': 'school',
  'sequence': 'i went to school this morning.'},
 {'score': 0.06230570003390312,
  'token': 3637,
  'token_str': 'sleep',
  'sequence': 'i went to sleep this morning.'},
 {'score': 0.04695269465446472,
  'token': 2465,
  'token_str': 'class',
  'sequence': 'i went to class this morning.'}]

# 한국어 BERT의 마스크드 언어 모델

In [ ]:
from transformers import TFBertForMaskedLM
from transformers import AutoTokenizer


In [ ]:
model = TFBertForMaskedLM.from_pretrained('klue/bert-base', from_pt=True)
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")


config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForMaskedLM: ['bert.embeddings.position_ids', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing TFBertForMaskedLM from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForMaskedLM from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForMaskedLM were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
inputs = tokenizer('축구는 정말 재미있는 [MASK]다.', return_tensors='tf')


In [ ]:
print(inputs['input_ids'])


tf.Tensor([[   2 4713 2259 3944 6001 2259    4  809   18    3]], shape=(1, 10), dtype=int32)


In [ ]:
print(inputs['token_type_ids'])


tf.Tensor([[0 0 0 0 0 0 0 0 0 0]], shape=(1, 10), dtype=int32)


In [ ]:
print(inputs['attention_mask'])


tf.Tensor([[1 1 1 1 1 1 1 1 1 1]], shape=(1, 10), dtype=int32)


In [ ]:
from transformers import FillMaskPipeline
pip = FillMaskPipeline(model=model, tokenizer=tokenizer)


TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
Device set to use 0


In [ ]:
pip('축구는 정말 재미있는 [MASK]다.')


[{'score': 0.8963508605957031,
  'token': 4559,
  'token_str': '스포츠',
  'sequence': '축구는 정말 재미있는 스포츠 다.'},
 {'score': 0.025957655161619186,
  'token': 568,
  'token_str': '거',
  'sequence': '축구는 정말 재미있는 거 다.'},
 {'score': 0.010033962316811085,
  'token': 3682,
  'token_str': '경기',
  'sequence': '축구는 정말 재미있는 경기 다.'},
 {'score': 0.007924417965114117,
  'token': 4713,
  'token_str': '축구',
  'sequence': '축구는 정말 재미있는 축구 다.'},
 {'score': 0.007844205014407635,
  'token': 5845,
  'token_str': '놀이',
  'sequence': '축구는 정말 재미있는 놀이 다.'}]

In [ ]:
pip('어벤져스는 정말 재미있는 [MASK]다.')


[{'score': 0.83824223279953,
  'token': 3771,
  'token_str': '영화',
  'sequence': '어벤져스는 정말 재미있는 영화 다.'},
 {'score': 0.02827564999461174,
  'token': 568,
  'token_str': '거',
  'sequence': '어벤져스는 정말 재미있는 거 다.'},
 {'score': 0.017189331352710724,
  'token': 4665,
  'token_str': '드라마',
  'sequence': '어벤져스는 정말 재미있는 드라마 다.'},
 {'score': 0.014989655464887619,
  'token': 3758,
  'token_str': '이야기',
  'sequence': '어벤져스는 정말 재미있는 이야기 다.'},
 {'score': 0.009382604621350765,
  'token': 4938,
  'token_str': '장소',
  'sequence': '어벤져스는 정말 재미있는 장소 다.'}]

In [ ]:
pip('나는 오늘 아침에 [MASK]에 출근을 했다.')


# 구글 BERT의 다음 문장 예측

In [3]:

import tensorflow as tf
from transformers import TFBertForNextSentencePrediction
from transformers import AutoTokenizer
#TFBertForNextSentencePrediction.from_pretrained('BERT 모델 이름')을 넣으면 두 개의 문장이 이어지는 문장 관계인지 여부를 판단하는 BERT 구조를 로드

In [5]:
from transformers import TFBertForNextSentencePrediction, AutoTokenizer
#TFBertForNextSentencePrediction.from_pretrained('BERT 모델 이름')을 넣으면 두 개의 문장이 이어지는 문장 관계인지 여부를 판단하는 BERT 구조를 로드
model = TFBertForNextSentencePrediction.from_pretrained(
    "bert-base-uncased",
    from_pt=True,              # PyTorch → TF 변환 로드
    use_safetensors=False      # safetensors 비활성화
)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
#AutoTokenizer.from_pretrained('모델 이름')을 넣으면 해당 모델이 학습되었을 당시에 사용되었던 토크나이저를 로드

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForNextSentencePrediction.

All the weights of TFBertForNextSentencePrediction were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForNextSentencePrediction for predictions without further training.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [6]:
prompt = "In Italy, pizza served in formal settings, such as at a restaurant, is presented unsliced."
next_sentence = "pizza is eaten with the use of a knife and fork. In casual settings, however, it is cut into wedges to be eaten while held in the hand."


In [7]:
encoding = tokenizer(prompt, next_sentence, return_tensors='tf')


TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.


In [8]:
print(encoding['input_ids'])


tf.Tensor(
[[  101  1999  3304  1010 10733  2366  1999  5337 10906  1010  2107  2004
   2012  1037  4825  1010  2003  3591  4895 14540  6610  2094  1012   102
  10733  2003  8828  2007  1996  2224  1997  1037  5442  1998  9292  1012
   1999 10017 10906  1010  2174  1010  2009  2003  3013  2046 17632  2015
   2000  2022  8828  2096  2218  1999  1996  2192  1012   102]], shape=(1, 58), dtype=int32)


In [9]:
print(tokenizer.cls_token, ':', tokenizer.cls_token_id)
print(tokenizer.sep_token, ':' , tokenizer.sep_token_id)


[CLS] : 101
[SEP] : 102


In [10]:
print(tokenizer.decode(encoding['input_ids'][0]))


[CLS] in italy, pizza served in formal settings, such as at a restaurant, is presented unsliced. [SEP] pizza is eaten with the use of a knife and fork. in casual settings, however, it is cut into wedges to be eaten while held in the hand. [SEP]


In [11]:
print(encoding['token_type_ids'])
'''
0이 연속적으로 등장하다가 어느 순간부터 1이 연속적으로 등장하는데, 이는 [CLS] 토큰의 위치부터 첫번째 문장이 끝나고나서 등장한 [SEP] 토큰까지의 위치에는 0이 등장하고, 다음 두번째 문장부터는 1이 등장하는 것입니다. token_type_ids에서는 0과 1로 두 개의 문장을 구분하고 있습니다.
'''

tf.Tensor(
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]], shape=(1, 58), dtype=int32)


In [12]:
logits = model(encoding['input_ids'], token_type_ids=encoding['token_type_ids'])[0]
softmax = tf.keras.layers.Softmax()
probs = softmax(logits)
print(probs)


tf.Tensor([[9.9999714e-01 2.8381855e-06]], shape=(1, 2), dtype=float32)


In [13]:
print('최종 예측 레이블 :', tf.math.argmax(probs, axis=-1).numpy())
'''
최종 예측 레이블은 0입니다. 이는 BERT가 다음 문장 예측을 학습했을 당시에 실질적으로 이어지는 두 개의 문장의 레이블은 0. 이어지지 않는 두 개의 문장의 경우에는 레이블을 1로 두고서 이진 분류로 학습을 하였기 때문입니다
'''

최종 예측 레이블 : [0]


In [14]:
# 상관없는 두 개의 문장
prompt = "In Italy, pizza served in formal settings, such as at a restaurant, is presented unsliced."
next_sentence = "The sky is blue due to the shorter wavelength of blue light."
encoding = tokenizer(prompt, next_sentence, return_tensors='tf')

logits = model(encoding['input_ids'], token_type_ids=encoding['token_type_ids'])[0]

softmax = tf.keras.layers.Softmax()
probs = softmax(logits)
print('최종 예측 레이블 :', tf.math.argmax(probs, axis=-1).numpy())


최종 예측 레이블 : [1]


# 한국어 BERT의 다음 문장 예측

In [15]:
import tensorflow as tf
from transformers import TFBertForNextSentencePrediction
from transformers import AutoTokenizer
#TFBertForNextSentencePrediction.from_pretrained('BERT 모델 이름')을 넣으면 두 개의 문장이 이어지는 문장 관계인지 여부를 판단하는 BERT 구조를 로드

In [16]:
from transformers import TFBertForNextSentencePrediction, AutoTokenizer
#TFBertForNextSentencePrediction.from_pretrained('BERT 모델 이름')을 넣으면 두 개의 문장이 이어지는 문장 관계인지 여부를 판단하는 BERT 구조를 로드
model = TFBertForNextSentencePrediction.from_pretrained(
    'klue/bert-base',
    from_pt=True,              # PyTorch → TF 변환 로드
    use_safetensors=False      # safetensors 비활성화
)
tokenizer = AutoTokenizer.from_pretrained('klue/bert-base')
#AutoTokenizer.from_pretrained('모델 이름')을 넣으면 해당 모델이 학습되었을 당시에 사용되었던 토크나이저를 로드

config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForNextSentencePrediction: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForNextSentencePrediction from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForNextSentencePrediction from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForNextSentencePrediction were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForNextSentencePrediction for predictions without further training.


tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [17]:
# 이어지는 두 개의 문장
prompt = "2002년 월드컵 축구대회는 일본과 공동으로 개최되었던 세계적인 큰 잔치입니다."
next_sentence = "여행을 가보니 한국의 2002년 월드컵 축구대회의 준비는 완벽했습니다."
encoding = tokenizer(prompt, next_sentence, return_tensors='tf')

logits = model(encoding['input_ids'], token_type_ids=encoding['token_type_ids'])[0]

softmax = tf.keras.layers.Softmax()
probs = softmax(logits)
print('최종 예측 레이블 :', tf.math.argmax(probs, axis=-1).numpy())


최종 예측 레이블 : [0]


In [18]:
# 상관없는 두 개의 문장
prompt = "2002년 월드컵 축구대회는 일본과 공동으로 개최되었던 세계적인 큰 잔치입니다."
next_sentence = "극장가서 로맨스 영화를 보고싶어요"
encoding = tokenizer(prompt, next_sentence, return_tensors='tf')

logits = model(encoding['input_ids'], token_type_ids=encoding['token_type_ids'])[0]

softmax = tf.keras.layers.Softmax()
probs = softmax(logits)
print('최종 예측 레이블 :', tf.math.argmax(probs, axis=-1).numpy())


최종 예측 레이블 : [1]


# BERT의 문장 임베딩(SBERT)을 이용한 한국어 챗봇

In [19]:
!pip install sentence_transformers


In [20]:
import numpy as np
import pandas as pd
from numpy import dot
from numpy.linalg import norm
import urllib.request
from sentence_transformers import SentenceTransformer


In [21]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv", filename="ChatBotData.csv")
train_data = pd.read_csv('ChatBotData.csv')
train_data.head()


,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [22]:
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')
#문장 임베딩을 얻기 위해서 사전 훈련된 BERT를 로드
'''
모델의 이름은 'xlm-r-100langs-bert-base-nli-stsb-mean-tokens'인데 이름이 의미하는 바는 100가지 언어를 지원(한국어 포함)하는 다국어 BERT BASE 모델로 SNLI 데이터를 학습 후 STS-B 데이터로 학습되었으며, 문장 표현을 얻기 위해서는 평균 풀링(mean-tokens)을 사용했다는 의미
다시 말해서 NLI 데이터를 학습 후에 STS 데이터로 추가 파인 튜닝한 모델
'''

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/731 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/527 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [23]:
train_data['embedding'] = train_data.apply(lambda row: model.encode(row.Q), axis = 1)


In [24]:
def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))


In [26]:
def return_answer(question):
    embedding = model.encode(question)
    train_data['score'] = train_data.apply(lambda x: cos_sim(x['embedding'], embedding), axis=1)
    return train_data.loc[train_data['score'].idxmax()]['A']


In [27]:
return_answer('결혼하고싶어')


'좋은 사람이랑 결혼할 수 있을 거예요.'

In [28]:
return_answer('나랑 커피먹을래?')


'카페인이 필요한 시간인가 봐요.'

In [30]:
return_answer('퇴근하고싶어')

'마음을 더 열어보세요.'